In [2]:
"""
把網頁抓下來，存到檔案裡面
"""

import requests

url = 'https://right-pet.cc/hospital.php?city=74'

resp = requests.get(url)

with open('/tmp/taipei_hospitals.html', 'w') as f:
    f.write(resp.text)

In [3]:
"""
把檔案讀出來
"""

with open('/tmp/taipei_hospitals.html') as f:
    html = f.read()

In [28]:
from bs4 import BeautifulSoup

In [29]:
soup = BeautifulSoup(html, 'lxml')

In [171]:
import re

def parse_hospital(hospital):
    """
        args:
            hospital<Tag>: hospital的beautifulsoup Tag object
        return:
            hospotal_dict<dict>: 整理過後的dict
    """
    hospital_dict = {}
    
    """
    使用regex擷取出hospital id與data
    """
    data_list = re.findall('<a href="hospital_detail.php\?hospitalSN=(\d+?)">(.+?)</a>', str(hospital))
    
    """
    使用regex擷取出經緯度
    """
    lat_lng = re.findall('''<a onclick="setMapToHospital\(1001, \'(.+?)\', \' (.+?)\'\)" title="查看地圖">''', str(hospital))
    
    hospital_dict['id']   = int(data_list[0][0])
    
    if lat_lng:
        hospital_dict['lng']  = lat_lng[0][1]
        hospital_dict['lat']  = lat_lng[0][0]
    else:
        hospital_dict['lng']  = ''
        hospital_dict['lat']  = ''
    hospital_dict['name'] = data_list[0][1]
    hospital_dict['area'] = data_list[1][1]
    hospital_dict['address'] = data_list[2][1]
    hospital_dict['tel'] = data_list[3][1]
    
    """
    Regex取出日期
    """
    hospital_dict['last_updated_at'] = re.findall('''\d\d\d\d-\d\d-\d\d''', data_list[4][1])[0]
    return hospital_dict

In [172]:
parse_hospital(soup.select('div#hospital_list_block')[0])

{'address': '新生南路1段144-1號1樓',
 'area': '台北市中正區',
 'id': 1001,
 'last_updated_at': '2012-10-16',
 'lat': '25.03569',
 'lng': '121.53247399999998',
 'name': '瀚生動物醫院',
 'tel': '02-23581765'}

In [173]:
hospitals = soup.select('div#hospital_list_block')

hospital_list = []
for hospital in hospitals:
    hospital_list.append(parse_hospital(hospital))

In [174]:
hospital_list[:5]

[{'address': '新生南路1段144-1號1樓',
  'area': '台北市中正區',
  'id': 1001,
  'last_updated_at': '2012-10-16',
  'lat': '25.03569',
  'lng': '121.53247399999998',
  'name': '瀚生動物醫院',
  'tel': '02-23581765'},
 {'address': '杭州南路 2 段 92 號 1 樓',
  'area': '台北市中正區',
  'id': 1000,
  'last_updated_at': '2012-10-16',
  'lat': '',
  'lng': '',
  'name': '博愛動物醫院',
  'tel': '(02)3393-8850'},
 {'address': '羅斯福路 2 段 138 號 1 樓',
  'area': '台北市中正區',
  'id': 999,
  'last_updated_at': '2012-10-16',
  'lat': '',
  'lng': '',
  'name': '古亭動物醫院',
  'tel': '(02)2369-3373 '},
 {'address': '忠孝東路 2 段 74 號 1 樓',
  'area': '台北市中正區',
  'id': 998,
  'last_updated_at': '2012-10-16',
  'lat': '',
  'lng': '',
  'name': '台灣動物醫院',
  'tel': '(02)2341-2128 '},
 {'address': '重慶南路 3 段 60~4 號 2 樓',
  'area': '台北市中正區',
  'id': 997,
  'last_updated_at': '2012-10-16',
  'lat': '',
  'lng': '',
  'name': '中正犬貓醫院',
  'tel': '(02)2305-4880 '}]

In [175]:
import pandas as pd

In [176]:
df = pd.DataFrame(hospital_list)

In [177]:
def format_tel(tel_str):
    if not tel_str.startswith('('):
        raw_tel = tel_str.replace('-','')
        new_tel_str = "({}){}-{}".format(raw_tel[0:2], raw_tel[2:6], raw_tel[6:])
        return new_tel_str
    else:
        return tel_str

In [178]:
format_tel('02-23581765')

'(02)2358-1765'

In [179]:
df['tel'] = df['tel'].apply(format_tel)

In [180]:
df.head()

,address,area,id,last_updated_at,lat,lng,name,tel
0,新生南路1段144-1號1樓,台北市中正區,1001,2012-10-16,25.03569,121.53247399999998,瀚生動物醫院,(02)2358-1765
1,杭州南路 2 段 92 號 1 樓,台北市中正區,1000,2012-10-16,,,博愛動物醫院,(02)3393-8850
2,羅斯福路 2 段 138 號 1 樓,台北市中正區,999,2012-10-16,,,古亭動物醫院,(02)2369-3373
3,忠孝東路 2 段 74 號 1 樓,台北市中正區,998,2012-10-16,,,台灣動物醫院,(02)2341-2128
4,重慶南路 3 段 60~4 號 2 樓,台北市中正區,997,2012-10-16,,,中正犬貓醫院,(02)2305-4880
